In [7]:
from pydantic import BaseModel, Field, ConfigDict, ValidationError

In [8]:
class Model(BaseModel):
    model_config = ConfigDict(populate_by_name = True)
    first_name : str = Field(validation_alias="FirstName")

In [12]:
m = Model(FirstName="Issac")
m

Model(first_name='Issac')

In [17]:
data = {"FirstName" : "Issacs"}
m = Model.model_validate(data)
m

Model(first_name='Issacs')

In [18]:
m.model_dump()

{'first_name': 'Issacs'}

In [19]:
m.model_dump(by_alias=True)

{'first_name': 'Issacs'}

In [20]:
class Model(BaseModel):
    model_config = ConfigDict(populate_by_name = True)
    first_name : str = Field(validation_alias="FirstName", alias="firstName")

In [21]:
m = Model(FirstName="Issac")
m

Model(first_name='Issac')

In [22]:
m.model_dump(by_alias=True)

{'firstName': 'Issac'}

In [27]:
class Model(BaseModel):
    model_config = ConfigDict(populate_by_name = True)
    first_name : str = Field(
        validation_alias="FirstName", 
        alias="firstName",
        serialization_alias="givenName")

In [28]:
data = {"FirstName" : "Issacs"}
m = Model.model_validate(data)
m

Model(first_name='Issacs')

In [29]:
Model(first_name='Issacs')

Model(first_name='Issacs')

In [30]:
m.model_dump(by_alias=True)

{'givenName': 'Issacs'}

In [33]:
from pydantic.alias_generators import to_camel

In [62]:
class Model(BaseModel):
    model_config = ConfigDict(populate_by_name = True, alias_generator = to_camel)
    first_name : str
    last_name: str

In [63]:
data = {"FirstName" : "Issacs", "lastName" : "Newton"}

In [64]:
Model.model_fields

{'first_name': FieldInfo(annotation=str, required=True, alias='firstName', alias_priority=1),
 'last_name': FieldInfo(annotation=str, required=True, alias='lastName', alias_priority=1)}

In [65]:
class Model(BaseModel):
    model_config = ConfigDict(populate_by_name = True, alias_generator = to_camel)
    first_name : str = Field(validation_alias = "FirstName", serialization_alias="givenName")
    last_name: str

In [66]:
data = {"FirstName" : "Issacs", "lastName" : "Newton"}
m = Model.model_validate(data)
m

Model(first_name='Issacs', last_name='Newton')

In [67]:
m.model_dump(by_alias=True)

{'givenName': 'Issacs', 'lastName': 'Newton'}

In [72]:
from pydantic import AliasChoices

In [73]:
class Model(BaseModel):
    model_config = ConfigDict(populate_by_name = True, alias_generator = to_camel)
    first_name : str = Field(
        validation_alias = AliasChoices("FirstName", "GivenName"),
        serialization_alias = "givenName"
    )
    last_name : str

In [74]:
data = {
    "FirstName": "Issac",
    "lastName" : "Newton"
}
m = Model.model_validate(data)
m

Model(first_name='Issac', last_name='Newton')

In [75]:
m.model_dump(by_alias=True)

{'givenName': 'Issac', 'lastName': 'Newton'}

In [84]:
data = {
    "databases" : {
        "redis" : {
            "name" : "Local Redis",
            "redis_conn" : "redis://secret@localhost:9000"
        },
        "pgsql" : {
            "name" : "Local Postgres",
            "pgsql_conn" : "postgres://secret@localhost:9000"
        },
        "nosql" : {
            "name" : "Local Mongodb",
            "mongo_conn" : "mongodb://secret@localhost:9000"
        }
    }
}

In [85]:
class Database(BaseModel):
    model_config = ConfigDict(populate_by_name = True, alias_generator = to_camel)
    name: str
    connection: str = Field(validation_alias = AliasChoices("redis_conn", "mongo_conn", "pgsql_conn"))

In [88]:
databases = {}

for key, value in data["databases"].items():
    m = Database.model_validate(value)
    databases[key] = m
    
databases

{'redis': Database(name='Local Redis', connection='redis://secret@localhost:9000'),
 'pgsql': Database(name='Local Postgres', connection='postgres://secret@localhost:9000'),
 'nosql': Database(name='Local Mongodb', connection='mongodb://secret@localhost:9000')}

In [90]:
class Databases(BaseModel):
    databases: dict[str, Database]

In [92]:
databases = Databases.model_validate(data)

In [93]:
databases

Databases(databases={'redis': Database(name='Local Redis', connection='redis://secret@localhost:9000'), 'pgsql': Database(name='Local Postgres', connection='postgres://secret@localhost:9000'), 'nosql': Database(name='Local Mongodb', connection='mongodb://secret@localhost:9000')})

In [94]:
print(databases.model_dump_json(indent=2))

{
  "databases": {
    "redis": {
      "name": "Local Redis",
      "connection": "redis://secret@localhost:9000"
    },
    "pgsql": {
      "name": "Local Postgres",
      "connection": "postgres://secret@localhost:9000"
    },
    "nosql": {
      "name": "Local Mongodb",
      "connection": "mongodb://secret@localhost:9000"
    }
  }
}
